In [ ]:
#using intel D435 realsense to test image_corrosion_seg
#model: corrosionsegment_trt.pth(20.3MB)
#output: [colorframe_image, seg_image]
#last updated: 11/10/2021

In [ ]:
#load the model
from torch2trt import TRTModule
import torch
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('corrosionsegment_trt.pth'))

In [ ]:
#import module for display
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg
import ipywidgets

In [1]:
#import key function.py
from D435_corrosion_util import *

In [ ]:
#display and create the image boxes
seg_image = ipywidgets.Image(format='jpeg', width=128, height=128)
#depthframe_image = ipywidgets.Image(format='jpeg', width=128, height=128)
colorframe_image = ipywidgets.Image(format='jpeg', width=128, height=128)
#display(widgets.VBox([widgets.HBox([depthframe_image, colorframe_image, seg_image]),]))
display(widgets.VBox([widgets.HBox([colorframe_image, seg_image]),]))

In [ ]:
#load images from D435
dc = DepthCamera()
#ret, depth_frame, color_frame = dc.get_frame()
ret, color_frame = dc.get_frame()

In [ ]:
#process for image segmentation of corrosion
import os
import cv2
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
from matplotlib import pyplot as plt

#from IPython.display import clear_output 

import random
from matplotlib.pyplot import figure, draw, pause
import numpy
%matplotlib inline 

#execution for segmentation
def execute(change):
    image = change['new']
    #ret, depth_frame, color_frame = dc.get_frame()
    ret, color_frame = dc.get_frame()
    output = model_trt(preprocess(color_frame).half())[0].detach().cpu().float().numpy()
    im=((output.squeeze())>0.6).astype(int)
    mask=decode_segmap(im)
    seg_image.value = bgr8_to_jpeg(mask)   
    # plt.imshow(im, cmap="Blues")
    # plt.show()
    # clear_output(wait=True)

    #image of D435 generation
    #depthframe_image.value = bgr8_to_jpeg(depth_frame)
    colorframe_image.value = bgr8_to_jpeg(color_frame)

In [ ]:
#run real time of the images
while True:
    mask = execute({'new': color_frame})

In [ ]:
#The following is the test

In [ ]:
#print(color_frame)
print(color_frame.size)
print(color_frame.shape)
print(color_frame.dtype)
print(color_frame.astype)
print(color_frame.ctypes)
print(color_frame.ndim)

In [ ]:
plt.savefig("color_frame.png")

In [ ]:
# #numpy.ndarray
# print(color_frame.size)
# print(color_frame.shape)
# print(color_frame.dtype)
# print(color_frame.astype)
# print(color_frame.ctypes)
# print(color_frame.ndim)

In [ ]:

# from jetbot import Camera
# color_frame.observe(execute, names='value')

In [ ]:
import cv2, PIL
import numpy as np
import torchvision
import torchvision.transforms as T

device = torch.device('cuda')
print(device)
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(image):
    x = image
    # x = cv2.resize(x, (128, 128))
#    x = cv2.center_crop(x, (224, 224))
    # x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

#torch_output = model(preprocess(image)).half()[0].detach().cpu().float().numpy()
trt_output = model_trt(preprocess(image)).half()[0].detach().cpu().float().numpy()
#print(torch_output.shape)
print(trt_output.shape)
# image_2 = preprocess(image)

In [ ]:
def load_one_image(image_path, image_size=128):

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image , (image_size , image_size)) # resizing before inserting to the network
    
    return image

image = load_one_image("1636511550059.png")

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline 


o = trt_output.squeeze()
o = o > 0.6
o = o.astype(int)
print(o.shape)
plt.imshow(o, cmap="Blues")
plt.show()